## Projet Bayésien
Guillaume et Yanis **<3**

## Introduction du problème 

Le problème développé dans ce projet est d'implémenter un échantillonneur selon le modèle statistique donné par le pack de données choisi. 
Nous travaillerons sur le dataset **Eyes**. Il s'agit d'un ensemble de mesures de longueurs d'ondes pour lesquelles les singes qui ont été testés ont une sensibilité maximale. Nos données sont donc en nanomètres.


Ce projet s'articulera en trois mouvements :
- Importation du dataset
- Etude approfondie des modèles statistiques à implémenter
- Implémentation d'un sampler de Gibbs pour la génération d'un échantillon suivant les loi données par le dataset **Eyes**. On implémentera un échantillonneur pour chacun des deux modèles présentés dans le dataset, afin de pouvoir se rendre compte de l'effet d'attraction de classe.

Importation des **Data** :

In [15]:
# Simulation et analyse des données
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

In [ ]:
# Conversion du fichier eyes.data.R en Python
N = 48
y = np.array([
    529, 530, 532, 533.1, 533.4, 533.6, 533.7, 534.1, 534.8, 535.3,
    535.4, 535.9, 536.1, 536.3, 536.4, 536.6, 537, 537.4, 537.5,
    538.3, 538.5, 538.6, 539.4, 539.6, 540.4, 540.8, 542, 542.8,
    543, 543.5, 543.8, 543.9, 545.3, 546.2, 548.8, 548.7, 548.9,
    549, 549.4, 549.9, 550.6, 551.2, 551.4, 551.5, 551.6, 552.8,
    552.9, 553.2
])
alpha = np.array([1.0, 1.0])

# Conversion du fichier eyes.init.R en Python
lambda_1 = 535
theta = 5
sigmasq = 10
p1 = 0.5


In [17]:
# Création d'un DataFrame pour analyser les données
df1 = pd.DataFrame({'y': y})
print(df1.describe())

                y
count   48.000000
mean   541.533333
std      7.022406
min    529.000000
25%    536.050000
50%    540.000000
75%    548.825000
max    553.200000


Dans l'analyse de l'équipe de Bowmaker et Al., les chercheurs ont retiré 500nm à toutes les valeurs des observations. Nous allons aussi procéder à ce prétraitement.

In [18]:
df = pd.DataFrame({'y': y-500})
print(df.describe()) 

               y
count  48.000000
mean   41.533333
std     7.022406
min    29.000000
25%    36.050000
50%    40.000000
75%    48.825000
max    53.200000


Ensuite, on passe y en format numpy pour l'exploitation dans l'échantillonneur

In [ ]:
y = df.to_numpy()

## Etude du modèle statistique non corrigé

DAG + lois conditionnelles (normalement c'est juste une gaussienne classique mais il faut prendre en plus le fait qu'on puisse changer de variable pour le lambda)

In [ ]:
def Eyes(n_iter, y, init, hyperparam):
    """Gibbs sampler for the Eyes dataset : 
        - n_iter is the length of the chain generated
        - y is the known data
        - init is the initialisation of the parameters we're estimating
        - hyperparam are the initialisations of the parameters of the laws we're estimating"""
    
    # On récupère les hyperparamètres de l'entrée

    [alpha, beta, mu_theta, sigma_theta, mu_lambda, sigma_lambda] = hyperparam

    # On crée les variables de stockage

    chaine = np.zeros(n_iter+1, len(init)) # stockage des paramètres
    chaine[0] = init
    latent = np.zeros(n_iter+1, len(hyperparam)) # stockage des hyperparamètres
    n = len(y)
    
    for k in range(1,n_iter+1):

        # Update des T_i
        for i in range(n):
            expo = -(lambda_1 + theta*(chaine[k-1, i]==2) - (mu_lambda + mu_theta))**2 / (2 * (sigma_lambda**2 + sigma_theta**2 * (chaine[k-1, i]==2)))
            p = 1 / np.sqrt(2*np.pi) / (sigma_lambda + sigma_theta) * np.exp(expo)
            print("p = ", p)
            chaine[k,i] = np.random.choice(a=[1,2], p=[1-p,p])

        # Update de lambda_1

        # Update de theta

        # Update de tau^2

        # Update de P

        # Mise à jour de la chaine 



In [ ]:
n_iter = 5

## Implémentation de l'échantillonneur de Gibbs pour le modèle initial

In [19]:
##def Gibbs_1

## Etude du modèle statistique corrigé

skibidi la suite

## Implémentation de la correction

In [20]:
##Au boulot